In [0]:
%run ./0.Config

In [0]:
dbutils.fs.rm("/dbfs/FileStore/tables/processed_vendors.csv",True)

In [0]:
@tool
def respond_to_vendor_rfq_submissions(task_description: str) -> str:
    """
    Processes RFQ submissions from vendors, generates email responses using LLM, 
    and sends them. Keeps track of processed submissions.
    """

    model = ChatDatabricks(endpoint="databricks-llama-4-maverick")
    email_prompt = PromptTemplate(
        input_variables=["vendor_name", "summary", "issue", "status"],
        template="""
You are a professional Procurement Manager replying to a vendor who submitted an RFQ response.
start with salutation
Vendor Name: {vendor_name}
Submission Summary (in ₹):
{summary}

Carefully check if any of the submitted rows contain issues:
    - Truck Count is 0 or negative
    - Price per Truck is ₹0.00 or negative
    - Total Cost is ₹0.00 or negative

If the submission is Valid:
- Thank the vendor.
- List out route-wise total number of trucks and total cost in a new indented line.
- Reassure them that their response is recorded.

If the submission status is Invalid:
- Mention where the problem is politely.
- Ask them to resubmit the form via: https://genai-automation-jkc3mhhnxkvlkh7ftab6lf.streamlit.app/

Vendor Name: {vendor_name}
Submission Summary: {summary}

Do not include subject in the body. Compose a short and professional email.

Keep the tone professional and concise.
Sign off with:

Best regards,  
Coca Cola Pvt Ltd  
procurement@cocacola.com
"""
    )
    chain = email_prompt | model

    # ------------------ READ DATA FROM ADLS ------------------
    file_path = "abfss://vendor-rfq@genaiautomationsa.dfs.core.windows.net/vendor_response/Vendor_Response.csv"
    df_spark = spark.read.option("header", "true").csv(file_path)
    df = df_spark.toPandas()
    df["Submitted At"] = pd.to_datetime(df["Submitted At"])

    # ------------------ LOAD TRACKER ------------------
    tracker_path = "/dbfs/FileStore/tables/processed_vendors.csv"

    tracker_df = pd.DataFrame(columns=["Vendor Name", "Vendor Email", "Submitted At"])
    try:
        tracker_df = pd.read_csv(tracker_path)
        tracker_df["Submitted At"] = pd.to_datetime(tracker_df["Submitted At"])
    except FileNotFoundError:
        tracker_df = pd.DataFrame(columns=["Vendor Name", "Vendor Email", "Submitted At"])

    # ------------------ VALIDATION FUNCTION ------------------
    def evaluate_submission(row):
        try:
            count = int(str(row["Count"]).strip())
            price = float(str(row["Price per Truck"]).strip())
            total = float(str(row["Total Cost"]).strip())

            if count <= 0:
                return "Invalid", f"Truck Count is 0 or negative in Route {row['Route ID']}"
            if price <= 0:
                return "Invalid", f"Price per Truck is 0 or negative in Route {row['Route ID']}"
            if total <= 0:
                return "Invalid", f"Total Cost is 0 or negative in Route {row['Route ID']}"

            return "Valid", ""
        except Exception as e:
            return "Invalid", f"Error parsing values for Route {row['Route ID']}: {e}"

    # ------------------ PROCESS NEW SUBMISSIONS ------------------
    grouped = df.groupby(["Vendor Name", "Vendor Email"])
    new_emails = []

    for (vendor_name, vendor_email), group in grouped:
        latest_time = group["Submitted At"].max()

        existing = tracker_df[tracker_df["Vendor Email"].str.lower() == vendor_email.lower()]
        if not existing.empty and latest_time <= existing["Submitted At"].values[0]:
            continue

        status, issue = "Valid", ""
        for _, row in group.iterrows():
            row_status, row_issue = evaluate_submission(row)
            if row_status == "Invalid":
                status, issue = row_status, row_issue
                break

        summary = group[["Route ID", "Truck Type", "Count", "Price per Truck", "Total Cost"]].copy()
        summary["Price per Truck"] = summary["Price per Truck"].astype(float).apply(lambda x: f"₹{x:,.2f}")
        summary["Total Cost"] = summary["Total Cost"].astype(float).apply(lambda x: f"₹{x:,.2f}")
        summary_text = summary.to_string(index=False)

        if status == "Valid":
            group["Count"] = group["Count"].astype(int)
            group["Total Cost"] = group["Total Cost"].astype(float)

            summary_grouped = group.groupby("Route ID").agg({
                "Count": "sum",
                "Total Cost": "sum"
            }).reset_index()

            route_summary_lines = [
                f"- Route {row['Route ID']}: {row['Count']} trucks, Total Cost: ₹{row['Total Cost']:,.2f}"
                for _, row in summary_grouped.iterrows()]
            summary_text += "\n\nRoute-wise Summary:\n" + "\n".join(route_summary_lines)

        email_content = chain.invoke({
            "vendor_name": vendor_name,
            "summary": summary_text,
            "issue": issue,
            "status": status
        })
        if hasattr(email_content, "content"):  
            output = email_content.content
        print(f"\n----- EMAIL TO SEND -----\nTo: {vendor_email}\n\n{output}\n")

        new_emails.append({
            "Vendor Name": vendor_name.title(),
            "Vendor Email": vendor_email.lower(),
            "Submitted At": latest_time,
            "Email Content": output
        })

    # ------------------ SEND EMAILS ------------------
    yag = yagmail.SMTP(EMAIL_USER, EMAIL_PASSWORD)
    for entry in new_emails:
        # UNCOMMENT TO SEND EMAILS
        yag.send(
            to=entry["Vendor Email"],
            subject="Response to Your RFQ Submission",
            contents=entry["Email Content"]
        )
        print(f"✅ Ready to send to {entry['Vendor Email']}")

    # ------------------ UPDATE TRACKER FILE ------------------
    if new_emails:
        new_df = pd.DataFrame(new_emails)[["Vendor Name", "Vendor Email", "Submitted At"]]
        updated_tracker = pd.concat([tracker_df, new_df])
        updated_tracker["Vendor Name"] = updated_tracker["Vendor Name"].str.title()
        updated_tracker["Vendor Email"] = updated_tracker["Vendor Email"].str.lower()
        updated_tracker.sort_values("Submitted At", ascending=False, inplace=True)
        updated_tracker.drop_duplicates(subset="Vendor Email", keep="first", inplace=True)
        updated_tracker.to_csv(tracker_path, index=False)
        print(f"📝 Tracker updated with {len(new_df)} new entries.")
    else:
        print("📭 No new submissions to process.")

    return f"✅ Processed {len(new_emails)} new submission(s)."

In [0]:
tools = [respond_to_vendor_rfq_submissions]

agent = initialize_agent(
    tools=tools,
    llm=ChatDatabricks(endpoint="databricks-llama-4-maverick"),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Send follow-up thank you emails after RFQ submission to the vendors.")